# Práctico 4

## Transform Pattern

**"Transform pattern"** en el contexto de Machine Learning se refiere a la técnica de manipular y cambiar los datos de entrada antes de que sean utilizados por un modelo. 

Estos cambios pueden ayudar a mejorar el rendimiento del modelo y a hacer que el modelo sea más robusto ante variaciones en los datos de entrada.

Las transformaciones se aplican típicamente durante la etapa de pre_procesamiento de los datos y pueden implicar muchas técnicas diferentes, dependiendo del tipo de datos y del problema que se está tratando de resolver.

In [1]:
!pip install tensorflow==2.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 51.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 28.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.2 MB/s eta 0:00:00:00:01
   

In [2]:
import tensorflow as tf
print(tf.__version__)


2024-06-18 04:15:27.335301: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 04:15:27.459104: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 04:15:27.695655: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 04:15:28.957807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.16.1


In [3]:
%%time 

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Resizing, Rescaling
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

# Cargar y normalizar el conjunto de datos 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

data_preprocessing = Sequential([
    Resizing(32, 32),  # Redimensionar a 32x32
    Rescaling(1./255)  # Normalización adicional después del redimensionamiento
])

# Define tu modelo
model = tf.keras.models.Sequential([
    data_preprocessing,

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),  # Ajustar a 3 canales de entrada
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compila y entrena el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=256, epochs=5, validation_data=(x_test, y_test))

# Guarda el modelo
model.save('transform_pattern_conv.h5')


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


2024-06-18 04:15:46.856647: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 153600000 exceeds 10% of free system memory.
2024-06-18 04:15:50.122193: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-06-18 04:15:50.424928: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-06-18 04:15:50.451427: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-06-18 04:15:50.471791: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.


196/196 ━━━━━━━━━━━━━━━━━━━━ 180s 899ms/step - accuracy: 0.3099 - loss: 2.2269 - val_accuracy: 0.1518 - val_loss: 3.2739
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 199s 888ms/step - accuracy: 0.5159 - loss: 1.3592 - val_accuracy: 0.2643 - val_loss: 2.2165
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 204s 897ms/step - accuracy: 0.5868 - loss: 1.1640 - val_accuracy: 0.6002 - val_loss: 1.1095
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 176s 899ms/step - accuracy: 0.6342 - loss: 1.0382 - val_accuracy: 0.6745 - val_loss: 0.9007
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 202s 898ms/step - accuracy: 0.6590 - loss: 0.9733 - val_accuracy: 0.7128 - val_loss: 0.8062


CPU times: user 24min 54s, sys: 1min 49s, total: 26min 43s
Wall time: 16min 36s


En este caso, las transformaciones de datos **(Resizing y Rescaling)** se definen como capas Keras y se agregan al inicio de tu modelo. 

Esto significa que estas transformaciones se aplicarán automáticamente a las imágenes a medida que pasen por el modelo, ya sea durante el entrenamiento o durante la inferencia. 

Además, como las capas de preprocesamiento son parte del modelo, se guardarán junto con el modelo cuando lo guardes con model.save(). 

## Experiment tracking
### wandb: https://wandb.ai/site


Primero, vamos a agregar experiment tracking utilizando wandb (Weights & Biases). Esto nos va a permitir monitorear los experimentos en tiempo real, guardar nuestros modelos , resultados, y podremos compartir experimentos con otros.

[Wandb collab full explained notebook ](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb#scrollTo=jufPgkgqz2eF)

### 👟 Run an experiment 

1.  **Start a new run** and pass in hyperparameters to track

2.  **Log metrics** from training or evaluation

3.  **Visualize results** in the dashboard

4. **Generate alerts** in the dashboard 

In [4]:
!pip install wandb==0.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 7.9 MB/s eta 0:00:00:00:01


In [5]:
import wandb

print("wandb version:", wandb.__version__)


wandb version: 0.17.0


In [6]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import wandb
from wandb.integration.keras import WandbCallback

# Configura tu clave API directamente
wandb.login(key='e6c3f52fd7f2cb748d081dbb102141c3fcddf655')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/codespace/.netrc


True

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Resizing, Rescaling
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt
import random


# Definir un callback personalizado para wandb
class CustomWandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs, step=epoch)

# Launch 2 experiments, trying different dropout rates
for run in range(2):
    
    # Start a run, tracking hyperparameters
    wandb.init(
        project="ml-en-produccion",
        config={
            "activation_1": "relu",
            "dropout": random.uniform(0.01, 0.80),
            "optimizer": "adam",
            "loss": "categorical_crossentropy",
            "metric": "accuracy",
            "epoch": 3,
            "batch_size": 512,
        },
    )
    config = wandb.config
    
    # Cargar y normalizar el conjunto de datos 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Convertir las etiquetas en one-hot
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    # Define el preprocesamiento de la imagen
    data_preprocessing = Sequential([
        Resizing(32, 32),  # Redimensionar a 32x32
        Rescaling(1./255)  # Normalización adicional después del redimensionamiento
    ])

    # Define tu modelo
    model = tf.keras.models.Sequential([
        data_preprocessing,  

        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(config.dropout),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(config.dropout),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compila y entrena el modelo
    model.compile(optimizer=config.optimizer, loss=config.loss, metrics=[config.metric])

    # Add CustomWandbCallback to log metrics
    custom_wandb_callback = CustomWandbCallback()

    history = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epoch, 
                        validation_data=(x_test, y_test), callbacks=[custom_wandb_callback])
    
    wandb.finish()
    
    # Guardar el modelo
    model.save(f'model_run_{run}.h5')


wandb: Currently logged in as: martin-martinez-fontoura (ort-martin). Use `wandb login --relogin` to force relogin


Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.2737 - loss: 2.3881 - val_accuracy: 0.0976 - val_loss: 3.0330
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.4545 - loss: 1.5434 - val_accuracy: 0.1445 - val_loss: 3.3377
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.5373 - loss: 1.3011 - val_accuracy: 0.1244 - val_loss: 3.5202


accuracy,▁▆█
loss,█▃▁
val_accuracy,▁█▅
val_loss,▁▅█
accuracy,0.54986
loss,1.26733
val_accuracy,0.1244
val_loss,3.52016


Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.1812 - loss: 3.4643 - val_accuracy: 0.1000 - val_loss: 2.9645
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.3051 - loss: 2.0402 - val_accuracy: 0.1204 - val_loss: 2.7547
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.3585 - loss: 1.7658 - val_accuracy: 0.1683 - val_loss: 2.6656


accuracy,▁▅█
loss,█▂▁
val_accuracy,▁▃█
val_loss,█▃▁
accuracy,0.37278
loss,1.71189
val_accuracy,0.1683
val_loss,2.66564



##  W&B Alerts

**[W&B Alerts](https://docs.wandb.ai/guides/track/alert)** allows you to send alerts, triggered from your Python code, to your Slack or email. There are 2 steps to follow the first time you'd like to send a Slack or email alert, triggered from your code:

1) Turn on Alerts in your W&B [User Settings](https://wandb.ai/settings)

2) Add `wandb.alert()` to your code:

```python
wandb.alert(
    title="Low accuracy", 
    text=f"Accuracy is below the acceptable threshold"
)
```

In [8]:
import random 

# Start a wandb run
wandb.init(project="alerts-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f"Accuracy is: {accuracy}, {acc_threshold}")

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title="Low Accuracy",
            text=f"Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}",
        )
        print("Alert triggered")
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 1.16, 0.3
Accuracy is: 0.818, 0.3
Accuracy is: 0.696, 0.3
Accuracy is: 0.647, 0.3
Accuracy is: 1.382, 0.3
Accuracy is: 1.542, 0.3
Accuracy is: 1.087, 0.3
Accuracy is: 1.529, 0.3
Accuracy is: 1.048, 0.3
Accuracy is: 0.886, 0.3
Accuracy is: 0.704, 0.3
Accuracy is: 0.184, 0.3
Alert triggered


Accuracy,▆▄▄▃▇█▆█▅▅▄▁
Accuracy,0.184


## H Tuning - wandb

In [9]:
import os
import wandb
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Cargar y normalizar el conjunto de datos
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Usar solo una muestra del conjunto de datos de prueba
x_test_small = x_test[:1]
y_test_small = to_categorical(y_test[:1], 10)

# Definir el callback personalizado para wandb
class CustomWandbCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs, step=epoch)

# Definir la configuración del sweep
sweep_config = {
    'method': 'grid',  # el método de búsqueda de hiperparámetros
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'  
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01]  # Un solo valor para minimizar el tiempo de ejecución
        },
        'batch_size': {
            'values': [64]  # Un solo valor para minimizar el tiempo de ejecución
        },
        'run_index': {
            'values': [0, 1]  # Índices de los modelos a cargar
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="Htuning")

# Definir la función de entrenamiento
def train():
    run = wandb.init()
    config = run.config
    
    print("Configuración de la ejecución:", config)
    
    # Mostrar archivos en el directorio actual
    print("Archivos en el directorio actual:", os.listdir('.'))
    
    # Cargar el modelo basado en el índice del run
    run_index = config.run_index
    model_path = f"model_run_{run_index}.h5"
    
    if os.path.exists(model_path):
        print(f"Cargando el modelo desde {model_path}")
        model = load_model(model_path)
    else:
        raise FileNotFoundError(f"Archivo de modelo {model_path} no encontrado.")
    
    print(f"Evaluando el modelo {model_path}")
    
    # Evaluar el modelo en los datos de prueba reducidos
    loss, accuracy = model.evaluate(x_test_small, y_test_small, verbose=0)
    
    # Loguear los resultados en wandb
    wandb.log({"val_loss": loss, "val_accuracy": accuracy})
    print(f"Modelo {model_path} - Pérdida de validación: {loss}, Precisión de validación: {accuracy}")
    
    wandb.finish()
    print("Ejecución de Wandb finalizada")

# Ejecutar el agente
print("Iniciando el sweep...")
wandb.agent(sweep_id, function=train)
print("Sweep finalizado")


Create sweep with ID: f2mvj1md
Sweep URL: https://wandb.ai/ort-martin/Htuning/sweeps/f2mvj1md
Iniciando el sweep...


wandb: Agent Starting Run: c4xjo9yy with config:
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.01
wandb: 	run_index: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Configuración de la ejecución: {'batch_size': 64, 'learning_rate': 0.01, 'run_index': 0}
Archivos en el directorio actual: ['wandb', 'practico_4.ipynb', 'model_run_0.h5', 'transform_pattern_conv.h5', '.git', 'README.md', 'model_run_1.h5']
Cargando el modelo desde model_run_0.h5


Evaluando el modelo model_run_0.h5
Modelo model_run_0.h5 - Pérdida de validación: 3.0713162422180176, Precisión de validación: 0.0


val_accuracy,▁
val_loss,▁
val_accuracy,0.0
val_loss,3.07132


...

Configuración de la ejecución: {'batch_size': 64, 'learning_rate': 0.01, 'run_index': 1}
Archivos en el directorio actual: ['wandb', 'practico_4.ipynb', 'model_run_0.h5', 'transform_pattern_conv.h5', '.git', 'README.md', 'model_run_1.h5']
Cargando el modelo desde model_run_1.h5
Evaluando el modelo model_run_1.h5
Modelo model_run_1.h5 - Pérdida de validación: 2.6014018058776855, Precisión de validación: 0.0


val_accuracy,▁
val_loss,▁
val_accuracy,0.0
val_loss,2.6014


Ejecución de Wandb finalizada


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Sweep finalizado
